# The Contracting Standard Library
There are unique variables and functions available for smart contracts at runtime. In the last notebook, we used the ORM variables that are provided via the `stdlib` to interact with the database. We will explore the other methods in this notebook and the concept of `environment` and `ctx`.

To see the basic standard library included at runtime, use `gather()` from the `env` module.

In [1]:
from contracting.stdlib import env

In [2]:
env.gather()

{'Variable': contracting.db.orm.Variable,
 'Hash': contracting.db.orm.Hash,
 'ForeignVariable': contracting.db.orm.ForeignVariable,
 'ForeignHash': contracting.db.orm.ForeignHash,
 '__Contract': contracting.db.contract.Contract,
 'sha3': <function contracting.stdlib.bridge.hashing.sha3(hex_str:str)>,
 'sha256': <function contracting.stdlib.bridge.hashing.sha256(hex_str:str)>,
 'datetime': contracting.stdlib.bridge.time.Datetime}

All of these functions are available at runtime. You can extend the standard library by updating the `env` dictionary. If you want to pass something more dynamic through, like the time that a transaction was submitted to the network, you would pass in an environment object.

In [3]:
def stdlib_environment_examples():
    
    @export
    def sha3_data(s):
        return sha3(s)
    
    @export
    def return_env_variable():
        return this_will_be_defined_later

First, let's try to access the `sha3` function exposed in the `stdlib`.

In [4]:
from contracting.client import ContractingClient
client = ContractingClient(signer='stu')
client.submit(stdlib_environment_examples)
contract = client.get_contract('stdlib_environment_examples')

In [5]:
contract.sha3_data(s='00ff00ff00ff')

'a0d5f1e1000980a0ae98cffb12072a41328bbdfebf3f6012aa021b428daea5b7'

Now, if we try to call `return_env_variable()`, we will get an error. This is because the variable is not included in the `stdlib` nor has been defined elsewhere in the contract.

In [6]:
try:
    contract.return_env_variable()
except Exception as e:
    print(e)

name 'this_will_be_defined_later' is not defined


But if we pass it in the environment, it becomes accessible. This function is used by our blockchain to pass contextual information such as block height, block hash, transaction time, etc.

In [7]:
environment = {'this_will_be_defined_later': 42}
contract.return_env_variable(environment=environment)

42

## Runtime Context `ctx`
At runtime, there is a constant defined called `ctx`. `ctx` contains three fields:

* `ctx.signer`

    This is the signer of the initial transaction. This variable never changes. It should not be used for access control.

---

* `ctx.caller`
    
    This is the direct caller of the function. As explained in the next notebook, smart contracts can import functions from other smart contracts. If you submit a transaction to a smart contract which calls upon another smart contract, the call stack is modified. The `caller` becomes the calling smart contract on a function. This should be used for access control.
    
---    
    
* `ctx.this`

    This is the name of the smart contract. This is how you will reference a smart contract for ascribing ownership to them.

In [8]:
def call_me():
    @export
    def caller():
        return ctx.caller
    
    @export
    def this():
        return ctx.this

def ctx_example():
    import call_me
    
    @export
    def ctx_now():
        return ctx.signer, ctx.caller, ctx.this
    
    @export
    def ctx_after_call():
        c = call_me.caller()
        t = call_me.this()
        
        return ctx.signer, c, t

In [9]:
client.submit(call_me)
client.submit(ctx_example)

In [10]:
ctx_contract = client.get_contract('ctx_example')

In [11]:
ctx_contract.ctx_now()

('stu', 'stu', 'ctx_example')

In [12]:
ctx_contract.ctx_after_call()

('stu', 'ctx_example', 'call_me')

Notice above how `ctx_after_call` returns different information. This is because `ctx` is modified after each function call. Because `ctx_example` called `call_me`, the `ctx.caller` returned was `ctx_example`. If we call that function directly, we will get `stu` back.

In [13]:
call_me_contract = client.get_contract('call_me')

In [14]:
call_me_contract.caller()

'stu'

## Why does `ctx` being dynamic matter?
Having `ctx` lets you create smart contracts that act as operators for users and other smart contracts. Because they are given their own identity, they are essentially the signers of their own function calls. This allows you to give them their own accounts, balances, ownerships, etc. to create structures that behave complexly and securely.

Assume you have a bank smart contract. You want to keep everyone's balance inside of the main bank vault, but keep sub-accounts in their name. You don't want the bank to be able to spend someone else's money on their behalf, so you would check the `ctx.caller` to make sure it is the user, and not the bank itself.

Let's make a token contract to demonstrate this idea.

In [15]:
def coin():
    balances = Hash()
    token_name = 'Stubucks'
    token_symbol = 'SBX'
    
    @construct
    def seed():
        # Whoever creates this smart contract is minted 1,000,000 tokens
        balances[ctx.caller] = 1000000
        
    @export
    def transfer(amount, to):
        # Make sure that the person calling this function has the amount they are trying to transfer
        assert balances[ctx.caller] >= amount, "You don't have enough to spend!"
        
        # If so, deduct from their account and send to who they want to send to
        balances[ctx.caller] -= amount
        balances[to] += amount
        
    @export
    def allow(amount, spender):
        # This creates a 'subaccount' to allow the spender to transfer from our account a certain amount
        balances[ctx.caller, spender] = amount
        
    @export
    def spend_on_behalf(amount, owner, to):
        # We make sure the subaccount has enough coins to spend
        assert balances[owner, ctx.caller] >= amount, "You can't spend that!"
        
        # If so, deduct from the amount that the subaccount can spend
        balances[owner, ctx.caller] -= amount
        
        # And then make the transfer
        balances[owner] -= amount
        balances[to] += amount
        
def bank():
    import coin
    
    balances = Hash()
    
    @export
    def deposit(amount):
        # The bank spends the coins it is allowed to on itself. It takes it from the subaccount and give it to itself
        # We don't need an assertion because this will fail
        coin.spend_on_behalf(amount=amount, owner=ctx.caller, to=ctx.this)
        
        # The account that of whoever called the deposit function is incremented accordingly
        balances[ctx.caller] += amount
        
    @export
    def withdraw(amount):
        # Make sure there is enough in the caller's account to withdraw
        assert balances[ctx.caller] >= amount, "You don't have enough in your account!"
        
        # Deduct from the account
        balances[ctx.caller] -= amount
        
        # Transfer the coins out to the caller
        coin.transfer(amount=amount, to=ctx.caller)

Here is the idea:

1. You must allow a user to spend tokens on your behalf by calling the `allow` function.
2. You can then deposit tokens into the bank by calling deposit after you have `allow`ed the bank to spend on your behalf.
3. You can withdraw tokens from the bank if they belong to you. However, the bank is the one that has the token balance. You have a sub-balance.
    
Let's see if it works!

In [16]:
client.submit(coin)
client.submit(bank)

coin_contract = client.get_contract('coin')
bank_contract = client.get_contract('bank')

coin_contract.balances['stu'] # Check if things @construct'ed appropriately

1000000

In [17]:
coin_contract.keys()

['coin.__author__',
 'coin.__code__',
 'coin.__compiled__',
 'coin.__type__',
 'coin.balances:stu']

In [18]:
print(coin_contract.__code__)

balances = Hash(contract='coin', name='balances')
token_name = 'Stubucks'
token_symbol = 'SBX'


def ____():
    balances[ctx.caller] = 1000000


def transfer(amount, to):
    assert balances[ctx.caller] >= amount, "You don't have enough to spend!"
    balances[ctx.caller] -= amount
    balances[to] += amount


def allow(amount, spender):
    balances[ctx.caller, spender] = amount


def spend_on_behalf(amount, owner, to):
    assert balances[owner, ctx.caller] >= amount, "You can't spend that!"
    balances[owner, ctx.caller] -= amount
    balances[owner] -= amount
    balances[to] += amount



In [19]:
coin_contract.transfer(amount=10, to='hi')
coin_contract.balances['hi']

10

In [20]:
# Let's verify that the bank has no coins
coin_contract.balances['bank']

0

In [21]:
coin_contract.allow(amount=500, spender='bank')

In [22]:
coin_contract.balances['stu', 'bank'] # This is how you access a 'subaccount.' A nice feature of multihashes!

500

In [23]:
coin_contract.balances['stu'] # Notice that it is not affecting the main account. It is just an allowance account.

999990

In [24]:
bank_contract.deposit(amount=450) # This should modify our balance and give bank 450 coins. Let's check!

In [25]:
stu_balance = coin_contract.balances['stu']
bank_balance = coin_contract.balances['bank']

# This should only be 50, because 450 were spent on the bank's behalf.
bank_allowance = coin_contract.balances['stu', 'bank']

print('stu balance: {}\nbank balance: {}\nbank allowance: {}'.format(
    stu_balance,
    bank_balance,
    bank_allowance
))

stu balance: 999540
bank balance: 450
bank allowance: 50


In [26]:
bank_contract.balances['stu'] # Our account in the bank reflects the total

450

## Adding another participant to the equation

Now, let's transfer some coins to another account and do the same thing to see how the bank's total account value goes up and is the sum of all of the subaccounts from under it. We will also withdraw our coins from the bank and see them reappear in our balance on the coin contract.

In [27]:
# 'stu' will transfer some coins to 'raghu' to put in the bank.
coin_contract.transfer(amount=5000, to='raghu')

In [28]:
# Remember, the default signer on this client is 'stu' so we have to set it to 'raghu' to call a function from him.
coin_contract.allow(amount=4000, spender='bank', signer='raghu')

In [29]:
raghu_balance = coin_contract.balances['raghu']
bank_raghu_allowance = coin_contract.balances['raghu', 'bank']

print('raghu balance: {}\nbank allowance for raghu: {}'.format(raghu_balance, bank_raghu_allowance))

raghu balance: 5000
bank allowance for raghu: 4000


In [30]:
# If raghu tries to deposit more than 4000, an error will occur
try:
    bank_contract.deposit(amount=4001, signer='raghu')
except AssertionError as e:
    print(e)

You can't spend that!


In [31]:
# Less than 4000 or 4000 will do.
bank_contract.deposit(amount=3999, signer='raghu')

In [32]:
# Updated balances
raghu_balance = coin_contract.balances['raghu']
bank_raghu_allowance = coin_contract.balances['raghu', 'bank']

print('raghu balance: {}\nbank allowance for raghu: {}'.format(raghu_balance, bank_raghu_allowance))

raghu balance: 1001
bank allowance for raghu: 1


In [33]:
# The bank will now have 450 + 3999 coins total, with two subaccounts.
bank_balance = coin_contract.balances['bank']
stu_bank_account = bank_contract.balances['stu']
raghu_bank_account = bank_contract.balances['raghu']

print('bank balance: {}\nstu bank account: {}\nraghu bank account: {}'.format(
    bank_balance,
    stu_bank_account,
    raghu_bank_account
))

bank balance: 4449
stu bank account: 450
raghu bank account: 3999


In [34]:
# If we try to withdraw more than our account balance from the bank, then we will have an AssertionError
try:
    bank_contract.withdraw(amount=500)
except AssertionError as e:
    print(e)

You don't have enough in your account!


In [35]:
stu_prior_balance = coin_contract.balances['stu']
bank_contract.withdraw(amount=100)
stu_after_balance = coin_contract.balances['stu']

print('stu prior balance: {}\nstu after balance: {}'.format(
    stu_prior_balance,
    stu_after_balance
))

stu prior balance:994540
stu after balance: 994640


Now that you understand the basics of the `ctx` object, you can create extremely robust smart contracts that pass around context. In the next section, we will learn more about the import system and multihashes to add even more capabilities to your smart contracts!